### Part 1. Algorithmic Implementation - BFGS Quasi-Newton Algorithm.

In [1]:
# Install libraries
%matplotlib inline
import numpy as np
from sympy import *
from scipy import stats
from scipy.optimize import fmin
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random
import time

### 1) func = x1^2 + x2^2 + x3^2      and     x0 = 1,1,1

In [2]:
################################################################################################
#                                        Trivariate Calculation                                #
################################################################################################

# Start at Example:
#x = np.matrix('1;1;1')

########################################## Objective ###########################################
# Define and symbolize objective function 
x1, x2, x3 = symbols('x1:4')
fn = x1**2 + x2**2 + x3**2

# Lambdify objective function
f = lambdify('x1, x2, x3', fn)

########################################## Derivative ###########################################

# Derivative
fnDiffx1 = fn.diff(x1)
fnDiffx2 = fn.diff(x2) 
fnDiffx3 = fn.diff(x3) 
fnDeriv = Matrix([[fnDiffx1], [fnDiffx2], [fnDiffx3]])

# Lambdify
fprime = lambdify('x1, x2, x3', fnDeriv)

########################################## Hessian    ###########################################

# Hessian
fnHessx11 = diff(diff(fn, x1),x1) ; fnHessx12 = diff(diff(fn, x1),x2) ; fnHessx13 = diff(diff(fn, x1),x3)
fnHessx21 = diff(diff(fn, x2),x1) ; fnHessx22 = diff(diff(fn, x2),x2) ; fnHessx23 = diff(diff(fn, x2),x3)
fnHessx31 = diff(diff(fn, x3),x1) ; fnHessx32 = diff(diff(fn, x3),x2) ; fnHessx33 = diff(diff(fn, x3),x3) 
fnHess = Matrix([[fnHessx11, fnHessx12, fnHessx13], [fnHessx21, fnHessx22, fnHessx23], [fnHessx31, fnHessx32, fnHessx33]])

# Lambdify
fHess = lambdify('x1, x2, x3', fnHess)

# Calculate Example
#f(x.item((0,0)) , x.item((1,0)) , x.item((2,0)))
#np.asmatrix(fprime(x.item((0,0)) , x.item((1,0)) , x.item((2,0))))
#np.asmatrix(fHess(x.item((0,0)) , x.item((1,0)) , x.item((2,0))))

In [3]:
# Start at
x = np.matrix('1;1;1')
print ("Initial Starting Point at:", x)

# Set convergence Precision and iteration limit
e = pow(10,-8)
itMax = 1000

# Assigning initial values
xold = np.matrix('0;0;0')
xnew = x
xlist, ylist = [xnew], [f(x.item((0,0)) , x.item((1,0)) , x.item((2,0)))]
Q = np.asmatrix(fHess(x.item((0,0)) , x.item((1,0)) , x.item((2,0))))
beta = np.identity(3)

# Loop Condition for next iteration
fprimeNorm = np.linalg.norm(np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0)) , xnew.item((2,0)))))
f_abs = abs(f(xnew.item((0,0)) , xnew.item((1,0)) , xnew.item((2,0))))

# Calculate runtime
starttime = time.time()

# Formula for BFGS
while fprimeNorm/(1 + f_abs) > e:
    xold = xnew
    fprimeVal = np.asmatrix(fprime(xold.item((0,0)) , xold.item((1,0)) , xold.item((2,0))))
        
    # Halting at itMax iterations to avoid convergence failure and computational overhead.    
    if (len(xlist) == itMax) :
        break;
    
    # Calculate direction
    direction = -1 * np.linalg.inv(beta) * fprimeVal
    
    # Calculate step size
    step = (-1 * np.transpose(direction) * fprimeVal)/ (np.transpose(direction) * Q * direction)
    
    # Calculate new x
    xnew = xold + step.item((0,0)) * direction
    
    # Update Secant Estimates for next iteration
    s = xnew - xold
    
    fprimeValNew = np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0)) , xnew.item((2,0))))
    y = fprimeValNew - fprimeVal
    
    beta_del = ((-1*(beta*s)*np.transpose(beta*s) / (np.transpose(s)*beta*s))) + ((y*np.transpose(y))/(np.transpose(y)*s))
    beta = beta + beta_del
    
    # Print search Dir, step length, new value: For first 10 and last 5 iterations
    if (len(xlist) <= 10):
        print("\n")
        print("Iteration: ",len(xlist))    
        print("Direction: ",direction)    
        print("Step Length: ",step.item((0,0)))  
        print("New Value: ",xnew)
        print("\n")
        
    if (len(xlist) > 10): 
        print(".", end = "")    
    
    xlist.append(xnew)
    ylist.append(f(xnew.item((0,0)) , xnew.item((1,0)) , xnew.item((2,0))))
    
    # Loop Condition for next iteration
    fprimeNorm = np.linalg.norm(np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0)) , xnew.item((2,0)))))
    f_abs = abs(f(xnew.item((0,0)) , xnew.item((1,0)) , xnew.item((2,0))))

runtime = time.time() - starttime 
print("\n")

if (len(xlist) == itMax):
    print ("No Solution has been found after %s iterations" %itMax)
    
else:
    print ("Local minimum occurs at:", xnew)
    print ("Number of steps:", len(xlist)-1)

print ("runtime: ", runtime)

Initial Starting Point at: [[1]
 [1]
 [1]]


Iteration:  1
Direction:  [[-2.]
 [-2.]
 [-2.]]
Step Length:  0.5
New Value:  [[ 0.]
 [ 0.]
 [ 0.]]




Local minimum occurs at: [[ 0.]
 [ 0.]
 [ 0.]]
Number of steps: 1
runtime:  0.008004903793334961


### 2) func = x1^2 + 2*x2^2 - 2*x1*x2 - 2*x2  and x0 = 0,0

In [4]:
################################################################################################
#                                        Bivariate Calculation                                 #
################################################################################################

# Start at Example:
#x = np.matrix('0;0')

########################################## Objective ###########################################
# Define and symbolize objective function 
x1, x2= symbols('x1:3')
fn = x1**2 + 2*x2**2 - 2*x1*x2 - 2*x2

# Lambdify objective function
f = lambdify('x1, x2', fn)

########################################## Derivative ###########################################

# Derivative
fnDiffx1 = fn.diff(x1)
fnDiffx2 = fn.diff(x2)  
fnDeriv = Matrix([[fnDiffx1], [fnDiffx2]])

# Lambdify
fprime = lambdify('x1, x2', fnDeriv)

########################################## Hessian    ###########################################

# Hessian
fnHessx11 = diff(diff(fn, x1),x1) ; fnHessx12 = diff(diff(fn, x1),x2)
fnHessx21 = diff(diff(fn, x2),x1) ; fnHessx22 = diff(diff(fn, x2),x2) 
fnHess = Matrix([[fnHessx11, fnHessx12], [fnHessx21, fnHessx22]])

# Lambdify
fHess = lambdify('x1, x2', fnHess)

# Calculate Example
#x = np.matrix('0;0')
#f(x.item((0,0)) , x.item((1,0)))
#np.asmatrix(fprime(x.item((0,0)) , x.item((1,0))))
#np.asmatrix(fHess(x.item((0,0)) , x.item((1,0))))

In [5]:
# Start at
x = np.matrix('0;0')
print ("Initial Starting Point at:", x)

# Set convergence Precision and iteration limit
e = pow(10,-8)
itMax = 1000

# Assigning initial values
xold = np.matrix('0;0')
xnew = x
xlist, ylist = [xnew], [f(x.item((0,0)) , x.item((1,0)))]
Q = np.asmatrix(fHess(x.item((0,0)) , x.item((1,0))))
beta = np.identity(2)

# Loop Condition for next iteration
fprimeNorm = np.linalg.norm(np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0)))))
f_abs = abs(f(xnew.item((0,0)) , xnew.item((1,0))))

# Calculate runtime
starttime = time.time()

# Formula for BFGS
while fprimeNorm/(1 + f_abs) > e:
    xold = xnew
    fprimeVal = np.asmatrix(fprime(xold.item((0,0)) , xold.item((1,0))))
        
    # Halting at itMax iterations to avoid convergence failure and computational overhead.    
    if (len(xlist) == itMax) :
        break;
    
    # Calculate direction
    direction = -1 * np.linalg.inv(beta) * fprimeVal
    
    # Calculate step size
    step = (-1 * np.transpose(direction) * fprimeVal)/ (np.transpose(direction) * Q * direction)
    
    # Calculate new x
    xnew = xold + step.item((0,0)) * direction
    
    # Update Secant Estimates for next iteration
    s = xnew - xold
    
    fprimeValNew = np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0))))
    y = fprimeValNew - fprimeVal
    
    beta_del = ((-1*(beta*s)*np.transpose(beta*s) / (np.transpose(s)*beta*s))) + ((y*np.transpose(y))/(np.transpose(y)*s))
    beta = beta + beta_del
    
    # Print search Dir, step length, new value: For first 10 and last 5 iterations
    if (len(xlist) <= 10):
        print("\n")
        print("Iteration: ",len(xlist))    
        print("Direction: ",direction)    
        print("Step Length: ",step.item((0,0)))  
        print("New Value: ",xnew)
        print("\n")
        
    if (len(xlist) > 10): 
        print(".", end = "")    
    
    xlist.append(xnew)
    ylist.append(f(xnew.item((0,0)) , xnew.item((1,0))))
    
    # Loop Condition for next iteration
    fprimeNorm = np.linalg.norm(np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0)))))
    f_abs = abs(f(xnew.item((0,0)) , xnew.item((1,0))))

runtime = time.time() - starttime 
print("\n")

if (len(xlist) == itMax):
    print ("No Solution has been found after %s iterations" %itMax)
    
else:
    print ("Local minimum occurs at:", xnew)
    print ("Number of steps:", len(xlist)-1)

print ("runtime: ", runtime)

Initial Starting Point at: [[0]
 [0]]


Iteration:  1
Direction:  [[ 0.]
 [ 2.]]
Step Length:  0.25
New Value:  [[ 0. ]
 [ 0.5]]




Iteration:  2
Direction:  [[ 1. ]
 [ 0.5]]
Step Length:  1.0
New Value:  [[ 1.]
 [ 1.]]




Local minimum occurs at: [[ 1.]
 [ 1.]]
Number of steps: 2
runtime:  0.012006521224975586


### 3) func = 100*(x2 - x1**2)^2 + (1-x1)^2 and x0 = -1.2,1
#### Notice this is a polynomial problem. Finding the Step size will be challenging as we cannot use Q anymore

In [6]:
################################################################################################
#                                        Bivariate Calculation                                 #
################################################################################################

# Start at Example:
#x = np.matrix('-1.2;1')

########################################## Objective ###########################################
# Define and symbolize objective function 
x1, x2= symbols('x1:3')
fn = 100*(x2 - x1**2)**2 + (1-x1)**2

# Lambdify objective function
f = lambdify('x1, x2', fn)

########################################## Derivative ###########################################

# Derivative
fnDiffx1 = fn.diff(x1)
fnDiffx2 = fn.diff(x2)  
fnDeriv = Matrix([[fnDiffx1], [fnDiffx2]])

# Lambdify
fprime = lambdify('x1, x2', fnDeriv)

########################################## Hessian    ###########################################

# Hessian
fnHessx11 = diff(diff(fn, x1),x1) ; fnHessx12 = diff(diff(fn, x1),x2)
fnHessx21 = diff(diff(fn, x2),x1) ; fnHessx22 = diff(diff(fn, x2),x2) 
fnHess = Matrix([[fnHessx11, fnHessx12], [fnHessx21, fnHessx22]])

# Lambdify
fHess = lambdify('x1, x2', fnHess)

# Calculate Example
#x = np.matrix('0;0')
#f(x.item((0,0)) , x.item((1,0)))
#np.asmatrix(fprime(x.item((0,0)) , x.item((1,0))))
#np.asmatrix(fHess(x.item((0,0)) , x.item((1,0))))

In [7]:
# Start at
x = np.matrix('-1.2;1')
print ("Initial Starting Point at:", x)

# Set convergence Precision and iteration limit
e = pow(10,-8)
itMax = 1000

# Assigning initial values
xold = np.matrix('0;0')
xnew = x
xlist, ylist = [xnew], [f(x.item((0,0)) , x.item((1,0)))]
beta = np.identity(2)

# Loop Condition for next iteration
fprimeNorm = np.linalg.norm(np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0)))))
f_abs = abs(f(xnew.item((0,0)) , xnew.item((1,0))))

# Function for fmin algorithm
def foptimize(x, direction):
    x = x.reshape((2,1))
    x = x + direction
    return (f(x.item((0,0)) , x.item((1,0))))

# Calculate runtime
starttime = time.time()

# Formula for BFGS
while fprimeNorm/(1 + f_abs) > e:
    xold = xnew
    fprimeVal = np.asmatrix(fprime(xold.item((0,0)) , xold.item((1,0))))
        
    # Halting at itMax iterations to avoid convergence failure and computational overhead.    
    if (len(xlist) == itMax) :
        break;
    
    # Calculate direction
    direction = -1 * np.linalg.inv(beta) * fprimeVal
    
    # Calculate step size
    # Use scipy fmin function to find ideal step size.
    step = fmin(foptimize, xold.reshape((1,2)) ,args = (direction))
    
    # Calculate new x
    xnew = xold + step * direction
    
    # Update Secant Estimates for next iteration
    s = xnew - xold
    
    fprimeValNew = np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0))))
    y = fprimeValNew - fprimeVal
    
    beta_del = ((-1*(beta*s)*np.transpose(beta*s) / (np.transpose(s)*beta*s))) + ((y*np.transpose(y))/(np.transpose(y)*s))
    beta = beta + beta_del
    
    # Print search Dir, step length, new value: For first 10 and last 5 iterations
    if (len(xlist) <= 10):
        print("\n")
        print("Iteration: ",len(xlist))    
        print("Direction: ",direction)    
        print("Step Length: ",step)  
        print("New Value: ",xnew)
        print("\n")
        
    if (len(xlist) > 10): 
        print(".", end = "")
    
    xlist.append(xnew)
    ylist.append(f(xnew.item((0,0)) , xnew.item((1,0))))
    
    # Loop Condition for next iteration
    fprimeNorm = np.linalg.norm(np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0)))))
    f_abs = abs(f(xnew.item((0,0)) , xnew.item((1,0))))

runtime = time.time() - starttime 
print("\n")

if (len(xlist) == itMax):
    print ("No Solution has been found after %s iterations" %itMax)
    
else:
    print ("Local minimum occurs at:", xnew)
    print ("Number of steps:", len(xlist)-1)

print ("runtime: ", runtime)

Initial Starting Point at: [[-1.2]
 [ 1. ]]


Iteration:  1
Direction:  [[ 215.6]
 [  88. ]]
Step Length:  [-150.69154271 -148.0192841 ]
New Value:  [[-45515.99360958]
 [-45513.79360958]]




Iteration:  2
Direction:  [[ 45514.79167629]
 [ 45690.78479004]]
Step Length:  [-45603.59346162 -45599.48622335]
New Value:  [[ -4.15915988e+09]
 [ -4.15915988e+09]]




Local minimum occurs at: [[ -4.15915988e+09]
 [ -4.15915988e+09]]
Number of steps: 2
runtime:  0.22716188430786133


### 4) func = (x1+x2)^4 + x2^2 and x0 = 2,-2
#### Notice this is a polynomial problem. Finding the Step size will be challenging as we cannot use Q anymore

In [8]:
################################################################################################
#                                        Bivariate Calculation                                 #
################################################################################################

# Start at Example:
#x = np.matrix('2,-2')

########################################## Objective ###########################################
# Define and symbolize objective function 
x1, x2= symbols('x1:3')
fn = (x1+x2)**4 + x2**2

# Lambdify objective function
f = lambdify('x1, x2', fn)

########################################## Derivative ###########################################

# Derivative
fnDiffx1 = fn.diff(x1)
fnDiffx2 = fn.diff(x2)  
fnDeriv = Matrix([[fnDiffx1], [fnDiffx2]])

# Lambdify
fprime = lambdify('x1, x2', fnDeriv)

########################################## Hessian    ###########################################

# Hessian
fnHessx11 = diff(diff(fn, x1),x1) ; fnHessx12 = diff(diff(fn, x1),x2)
fnHessx21 = diff(diff(fn, x2),x1) ; fnHessx22 = diff(diff(fn, x2),x2) 
fnHess = Matrix([[fnHessx11, fnHessx12], [fnHessx21, fnHessx22]])

# Lambdify
fHess = lambdify('x1, x2', fnHess)

# Calculate Example
#x = np.matrix('0;0')
#f(x.item((0,0)) , x.item((1,0)))
#np.asmatrix(fprime(x.item((0,0)) , x.item((1,0))))
#np.asmatrix(fHess(x.item((0,0)) , x.item((1,0))))

In [9]:
# Here step size will be calculated in a different way.
# Start at
x = np.matrix('2;-2')
print ("Initial Starting Point at:", x)

# Set convergence Precision and iteration limit
e = pow(10,-8)
itMax = 1000

# Assigning initial values
xold = np.matrix('0;0')
xnew = x
xlist, ylist = [xnew], [f(x.item((0,0)) , x.item((1,0)))]
beta = np.identity(2)

# Loop Condition for next iteration
fprimeNorm = np.linalg.norm(np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0)))))
f_abs = abs(f(xnew.item((0,0)) , xnew.item((1,0))))

# Function for fmin algorithm
def foptimize(x, direction):
    x = x.reshape((2,1))
    x = x + direction
    return (f(x.item((0,0)) , x.item((1,0))))

# Calculate runtime
starttime = time.time()

# Formula for BFGS
while fprimeNorm/(1 + f_abs) > e:
    xold = xnew
    fprimeVal = np.asmatrix(fprime(xold.item((0,0)) , xold.item((1,0))))
        
    # Halting at itMax iterations to avoid convergence failure and computational overhead.    
    if (len(xlist) == itMax) :
        break;
    
    # Calculate direction
    direction = -1 * np.linalg.inv(beta) * fprimeVal
    
    # Calculate step size
    # Use scipy fmin function to find ideal step size.
    step = fmin(foptimize, xold.reshape((1,2)) ,args = (direction))
    
    # Calculate new x
    xnew = xold + step * direction
    
    # Update Secant Estimates for next iteration
    s = xnew - xold
    
    fprimeValNew = np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0))))
    y = fprimeValNew - fprimeVal
    
    beta_del = ((-1*(beta*s)*np.transpose(beta*s) / (np.transpose(s)*beta*s))) + ((y*np.transpose(y))/(np.transpose(y)*s))
    beta = beta + beta_del
    
    # Print search Dir, step length, new value: For first 10 and last 5 iterations
    if (len(xlist) <= 10):
        print("\n")
        print("Iteration: ",len(xlist))    
        print("Direction: ",direction)    
        print("Step Length: ",step)  
        print("New Value: ",xnew)
        print("\n")
        
    if (len(xlist) > 10): 
        print(".", end = "")    
    
    xlist.append(xnew)
    ylist.append(f(xnew.item((0,0)) , xnew.item((1,0))))
    
    # Loop Condition for next iteration
    fprimeNorm = np.linalg.norm(np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0)))))
    f_abs = abs(f(xnew.item((0,0)) , xnew.item((1,0))))

runtime = time.time() - starttime 
print("\n")

if (len(xlist) == itMax):
    print ("No Solution has been found after %s iterations" %itMax)
    
else:
    print ("Local minimum occurs at:", xnew)
    print ("Number of steps:", len(xlist)-1)

print ("runtime: ", runtime)

Initial Starting Point at: [[ 2]
 [-2]]
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 72
         Function evaluations: 134


Iteration:  1
Direction:  [[ 0.]
 [ 4.]]
Step Length:  [-1.99998338 -2.00001663]
New Value:  [[ -6.00006652]
 [-10.00006652]]


Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 63
         Function evaluations: 116


Iteration:  2
Direction:  [[ 6.00104306]
 [ 9.99909092]]
Step Length:  [-8.00004649 -8.00008752]
New Value:  [[-134.00229252]
 [-138.00229252]]


Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 60
         Function evaluations: 110


Iteration:  3
Direction:  [[ 118.05429217]
 [ 138.00227666]]
Step Length:  [-128.02837009 -128.02819857]
New Value:  [[-32916.48378123]
 [-32920.48378123]]


Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 75
 

### 5.1) func = (x1-1)^2 + (x2-1)^2 + c*(x1^2 + x2^2 - 0.25)^2 and x0 = 1,-1 at c = 1
#### Notice this is a polynomial problem. Finding the Step size will be challenging as we cannot use Q anymore

In [10]:
################################################################################################
#                                        Bivariate Calculation                                 #
################################################################################################

# Start at Example:
#x = np.matrix('2,-2')

########################################## Objective ###########################################
# Define and symbolize objective function 
x1, x2= symbols('x1:3')

# at C=1
fn = (x1-1)**2 + (x2-1)**2 + 1*(x1**2 + x2**2 - 0.25)**2

# Lambdify objective function
f = lambdify('x1, x2', fn)

########################################## Derivative ###########################################

# Derivative
fnDiffx1 = fn.diff(x1)
fnDiffx2 = fn.diff(x2)  
fnDeriv = Matrix([[fnDiffx1], [fnDiffx2]])

# Lambdify
fprime = lambdify('x1, x2', fnDeriv)

########################################## Hessian    ###########################################

# Hessian
fnHessx11 = diff(diff(fn, x1),x1) ; fnHessx12 = diff(diff(fn, x1),x2)
fnHessx21 = diff(diff(fn, x2),x1) ; fnHessx22 = diff(diff(fn, x2),x2) 
fnHess = Matrix([[fnHessx11, fnHessx12], [fnHessx21, fnHessx22]])

# Lambdify
fHess = lambdify('x1, x2', fnHess)

# Calculate Example
#x = np.matrix('0;0')
#f(x.item((0,0)) , x.item((1,0)))
#np.asmatrix(fprime(x.item((0,0)) , x.item((1,0))))
#np.asmatrix(fHess(x.item((0,0)) , x.item((1,0))))

In [11]:
# Here step size will be calculated in a different way.
# Start at
x = np.matrix('1;-1')
print ("Initial Starting Point at:", x)

# Set convergence Precision and iteration limit
e = pow(10,-8)
itMax = 1000

# Assigning initial values
xold = np.matrix('0;0')
xnew = x
xlist, ylist = [xnew], [f(x.item((0,0)) , x.item((1,0)))]
beta = np.identity(2)

# Loop Condition for next iteration
fprimeNorm = np.linalg.norm(np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0)))))
f_abs = abs(f(xnew.item((0,0)) , xnew.item((1,0))))

# Function for fmin algorithm
def foptimize(x, direction):
    x = x.reshape((2,1))
    x = x + direction
    return (f(x.item((0,0)) , x.item((1,0))))

# Calculate runtime
starttime = time.time()

# Formula for BFGS
while fprimeNorm/(1 + f_abs) > e:
    xold = xnew
    fprimeVal = np.asmatrix(fprime(xold.item((0,0)) , xold.item((1,0))))
        
    # Halting at itMax iterations to avoid convergence failure and computational overhead.    
    if (len(xlist) == itMax) :
        break;
    
    # Calculate direction
    direction = -1 * np.linalg.inv(beta) * fprimeVal
    
    # Calculate step size
    # Use scipy fmin function to find ideal step size.
    step = fmin(foptimize, xold.reshape((1,2)) ,args = (direction))
    
    # Calculate new x
    xnew = xold + step * direction
    
    # Update Secant Estimates for next iteration
    s = xnew - xold
    
    fprimeValNew = np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0))))
    y = fprimeValNew - fprimeVal
    
    beta_del = ((-1*(beta*s)*np.transpose(beta*s) / (np.transpose(s)*beta*s))) + ((y*np.transpose(y))/(np.transpose(y)*s))
    beta = beta + beta_del
    
    # Print search Dir, step length, new value: For first 10 and last 5 iterations
    if (len(xlist) <= 10):
        print("\n")
        print("Iteration: ",len(xlist))    
        print("Direction: ",direction)    
        print("Step Length: ",step)  
        print("New Value: ",xnew)
        print("\n")
        
    if (len(xlist) > 10): 
        print(".", end = "")    
    
    xlist.append(xnew)
    ylist.append(f(xnew.item((0,0)) , xnew.item((1,0))))
    
    # Loop Condition for next iteration
    fprimeNorm = np.linalg.norm(np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0)))))
    f_abs = abs(f(xnew.item((0,0)) , xnew.item((1,0))))

runtime = time.time() - starttime 
print("\n")

if (len(xlist) == itMax):
    print ("No Solution has been found after %s iterations" %itMax)
    
else:
    print ("Local minimum occurs at:", xnew)
    print ("Number of steps:", len(xlist)-1)

print ("runtime: ", runtime)

Initial Starting Point at: [[ 1]
 [-1]]
Optimization terminated successfully.
         Current function value: 0.529336
         Iterations: 50
         Function evaluations: 97


Iteration:  1
Direction:  [[ -7.]
 [ 11.]]
Step Length:  [-1.71797818 -1.71794264]
New Value:  [[-5.87152176]
 [-7.87152176]]


Optimization terminated successfully.
         Current function value: 0.529336
         Iterations: 36
         Function evaluations: 70


Iteration:  2
Direction:  [[ -3.0778892 ]
 [ 14.35354463]]
Step Length:  [-5.35579924 -5.35577879]
New Value:  [[-66.26137509]
 [-68.26137509]]


Optimization terminated successfully.
         Current function value: 0.529336
         Iterations: 47
         Function evaluations: 91


Iteration:  3
Direction:  [[-292.24436618]
 [ 402.92010387]]
Step Length:  [-55.0558185  -55.05586414]
New Value:  [[-6159.62309039]
 [-6161.62309039]]


Optimization terminated successfully.
         Current function value: 0.529336
         Iterations: 53
        

### 5.2) func = (x1-1)^2 + (x2-1)^2 + c*(x1^2 + x2^2 - 0.25)^2 and x0 = 1,-1 at c = 10
#### Notice this is a polynomial problem. Finding the Step size will be challenging as we cannot use Q anymore

In [12]:
################################################################################################
#                                        Bivariate Calculation                                 #
################################################################################################

# Start at Example:
#x = np.matrix('2,-2')

########################################## Objective ###########################################
# Define and symbolize objective function 
x1, x2= symbols('x1:3')

# at C=10
fn = (x1-1)**2 + (x2-1)**2 + 10*(x1**2 + x2**2 - 0.25)**2

# Lambdify objective function
f = lambdify('x1, x2', fn)

########################################## Derivative ###########################################

# Derivative
fnDiffx1 = fn.diff(x1)
fnDiffx2 = fn.diff(x2)  
fnDeriv = Matrix([[fnDiffx1], [fnDiffx2]])

# Lambdify
fprime = lambdify('x1, x2', fnDeriv)

########################################## Hessian    ###########################################

# Hessian
fnHessx11 = diff(diff(fn, x1),x1) ; fnHessx12 = diff(diff(fn, x1),x2)
fnHessx21 = diff(diff(fn, x2),x1) ; fnHessx22 = diff(diff(fn, x2),x2) 
fnHess = Matrix([[fnHessx11, fnHessx12], [fnHessx21, fnHessx22]])

# Lambdify
fHess = lambdify('x1, x2', fnHess)

# Calculate Example
#x = np.matrix('0;0')
#f(x.item((0,0)) , x.item((1,0)))
#np.asmatrix(fprime(x.item((0,0)) , x.item((1,0))))
#np.asmatrix(fHess(x.item((0,0)) , x.item((1,0))))

In [13]:
# Here step size will be calculated in a different way.
# Start at
x = np.matrix('1;-1')
print ("Initial Starting Point at:", x)

# Set convergence Precision and iteration limit
e = pow(10,-8)
itMax = 1000

# Assigning initial values
xold = np.matrix('0;0')
xnew = x
xlist, ylist = [xnew], [f(x.item((0,0)) , x.item((1,0)))]
beta = np.identity(2)

# Loop Condition for next iteration
fprimeNorm = np.linalg.norm(np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0)))))
f_abs = abs(f(xnew.item((0,0)) , xnew.item((1,0))))

# Function for fmin algorithm
def foptimize(x, direction):
    x = x.reshape((2,1))
    x = x + direction
    return (f(x.item((0,0)) , x.item((1,0))))

# Calculate runtime
starttime = time.time()

# Formula for BFGS
while fprimeNorm/(1 + f_abs) > e:
    xold = xnew
    fprimeVal = np.asmatrix(fprime(xold.item((0,0)) , xold.item((1,0))))
        
    # Halting at itMax iterations to avoid convergence failure and computational overhead.    
    if (len(xlist) == itMax) :
        break;
    
    # Calculate direction
    direction = -1 * np.linalg.inv(beta) * fprimeVal
    
    # Calculate step size
    # Use scipy fmin function to find ideal step size.
    step = fmin(foptimize, xold.reshape((1,2)) ,args = (direction))
    
    # Calculate new x
    xnew = xold + step * direction
    
    # Update Secant Estimates for next iteration
    s = xnew - xold
    
    fprimeValNew = np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0))))
    y = fprimeValNew - fprimeVal
    
    beta_del = ((-1*(beta*s)*np.transpose(beta*s) / (np.transpose(s)*beta*s))) + ((y*np.transpose(y))/(np.transpose(y)*s))
    beta = beta + beta_del
    
    # Print search Dir, step length, new value: For first 10 and last 5 iterations
    if (len(xlist) <= 10):
        print("\n")
        print("Iteration: ",len(xlist))    
        print("Direction: ",direction)    
        print("Step Length: ",step)  
        print("New Value: ",xnew)
        print("\n")
        
    if (len(xlist) > 10): 
        print(".", end = "")    
    
    xlist.append(xnew)
    ylist.append(f(xnew.item((0,0)) , xnew.item((1,0))))
    
    # Loop Condition for next iteration
    fprimeNorm = np.linalg.norm(np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0)))))
    f_abs = abs(f(xnew.item((0,0)) , xnew.item((1,0))))

runtime = time.time() - starttime 
print("\n")

if (len(xlist) == itMax):
    print ("No Solution has been found after %s iterations" %itMax)
    
else:
    print ("Local minimum occurs at:", xnew)
    print ("Number of steps:", len(xlist)-1)

print ("runtime: ", runtime)

Initial Starting Point at: [[ 1]
 [-1]]
Optimization terminated successfully.
         Current function value: 0.768791
         Iterations: 54
         Function evaluations: 103


Iteration:  1
Direction:  [[-70.]
 [ 74.]]
Step Length:  [-1.79867486 -1.79877119]
New Value:  [[-6.20182729]
 [-8.20182729]]


Optimization terminated successfully.
         Current function value: 0.768791
         Iterations: 59
         Function evaluations: 112


Iteration:  2
Direction:  [[-74.31441447]
 [ 69.13964427]]
Step Length:  [ 2.78870424  2.7886707 ]
New Value:  [[-20.63505009]
 [-22.63505009]]


Optimization terminated successfully.
         Current function value: 0.768791
         Iterations: 70
         Function evaluations: 138


Iteration:  3
Direction:  [[-3035.30494029]
 [ 2815.93306371]]
Step Length:  [ 109.88724031  109.88727093]
New Value:  [[-24126.71894504]
 [-24128.71894504]]


Optimization terminated successfully.
         Current function value: 0.768791
         Iterations: 63

### 5.3) func = (x1-1)^2 + (x2-1)^2 + c*(x1^2 + x2^2 - 0.25)^2 and x0 = 1,-1 at c = 100
#### Notice this is a polynomial problem. Finding the Step size will be challenging as we cannot use Q anymore

In [14]:
################################################################################################
#                                        Bivariate Calculation                                 #
################################################################################################

# Start at Example:
#x = np.matrix('2,-2')

########################################## Objective ###########################################
# Define and symbolize objective function 
x1, x2= symbols('x1:3')

# at C=100
fn = (x1-1)**2 + (x2-1)**2 + 100*(x1**2 + x2**2 - 0.25)**2

# Lambdify objective function
f = lambdify('x1, x2', fn)

########################################## Derivative ###########################################

# Derivative
fnDiffx1 = fn.diff(x1)
fnDiffx2 = fn.diff(x2)  
fnDeriv = Matrix([[fnDiffx1], [fnDiffx2]])

# Lambdify
fprime = lambdify('x1, x2', fnDeriv)

########################################## Hessian    ###########################################

# Hessian
fnHessx11 = diff(diff(fn, x1),x1) ; fnHessx12 = diff(diff(fn, x1),x2)
fnHessx21 = diff(diff(fn, x2),x1) ; fnHessx22 = diff(diff(fn, x2),x2) 
fnHess = Matrix([[fnHessx11, fnHessx12], [fnHessx21, fnHessx22]])

# Lambdify
fHess = lambdify('x1, x2', fnHess)

# Calculate Example
#x = np.matrix('0;0')
#f(x.item((0,0)) , x.item((1,0)))
#np.asmatrix(fprime(x.item((0,0)) , x.item((1,0))))
#np.asmatrix(fHess(x.item((0,0)) , x.item((1,0))))

In [15]:
# Here step size will be calculated in a different way.
# Start at
x = np.matrix('1;-1')
print ("Initial Starting Point at:", x)

# Set convergence Precision and iteration limit
e = pow(10,-8)
itMax = 1000

# Assigning initial values
xold = np.matrix('0;0')
xnew = x
xlist, ylist = [xnew], [f(x.item((0,0)) , x.item((1,0)))]
beta = np.identity(2)

# Loop Condition for next iteration
fprimeNorm = np.linalg.norm(np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0)))))
f_abs = abs(f(xnew.item((0,0)) , xnew.item((1,0))))

# Function for fmin algorithm
def foptimize(x, direction):
    x = x.reshape((2,1))
    x = x + direction
    return (f(x.item((0,0)) , x.item((1,0))))

# Calculate runtime
starttime = time.time()

# Formula for BFGS
while fprimeNorm/(1 + f_abs) > e:
    xold = xnew
    fprimeVal = np.asmatrix(fprime(xold.item((0,0)) , xold.item((1,0))))
        
    # Halting at itMax iterations to avoid convergence failure and computational overhead.    
    if (len(xlist) == itMax) :
        break;
    
    # Calculate direction
    direction = -1 * np.linalg.inv(beta) * fprimeVal
    
    # Calculate step size
    # Use scipy fmin function to find ideal step size.
    step = fmin(foptimize, xold.reshape((1,2)) ,args = (direction))
    
    # Calculate new x
    xnew = xold + step * direction
    
    # Update Secant Estimates for next iteration
    s = xnew - xold
    
    fprimeValNew = np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0))))
    y = fprimeValNew - fprimeVal
    
    beta_del = ((-1*(beta*s)*np.transpose(beta*s) / (np.transpose(s)*beta*s))) + ((y*np.transpose(y))/(np.transpose(y)*s))
    beta = beta + beta_del
    
    # Print search Dir, step length, new value: For first 10 and last 5 iterations
    if (len(xlist) <= 10):
        print("\n")
        print("Iteration: ",len(xlist))    
        print("Direction: ",direction)    
        print("Step Length: ",step)  
        print("New Value: ",xnew)
        print("\n")
        
    if (len(xlist) > 10): 
        print(".", end = "")    
    
    xlist.append(xnew)
    ylist.append(f(xnew.item((0,0)) , xnew.item((1,0))))
    
    # Loop Condition for next iteration
    fprimeNorm = np.linalg.norm(np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0)))))
    f_abs = abs(f(xnew.item((0,0)) , xnew.item((1,0))))

runtime = time.time() - starttime 
print("\n")

if (len(xlist) == itMax):
    print ("No Solution has been found after %s iterations" %itMax)
    
else:
    print ("Local minimum occurs at:", xnew)
    print ("Number of steps:", len(xlist)-1)

print ("runtime: ", runtime)

Initial Starting Point at: [[ 1]
 [-1]]
Optimization terminated successfully.
         Current function value: 0.827655
         Iterations: 84
         Function evaluations: 154


Iteration:  1
Direction:  [[-700.]
 [ 704.]]
Step Length:  [-1.82010079 -1.82012951]
New Value:  [[-6.30062223]
 [-8.30062223]]


Optimization terminated successfully.
         Current function value: 0.827655
         Iterations: 77
         Function evaluations: 150


Iteration:  2
Direction:  [[-788.99224363]
 [ 614.4687322 ]]
Step Length:  [ 87.4416599   87.44161968]
New Value:  [[-15266.95086635]
 [-15268.95086635]]


Optimization terminated successfully.
         Current function value: 0.827655
         Iterations: 64
         Function evaluations: 122


Iteration:  3
Direction:  [[-28062.5 ]
 [ 58578.25]]
Step Length:  [-15257.69511193 -15257.69508708]
New Value:  [[ -4.65615275e+08]
 [ -4.65615277e+08]]




Local minimum occurs at: [[ -4.65615275e+08]
 [ -4.65615277e+08]]
Number of steps: 3
runtime: